# Pratiques d'écoutes musicales et omnivorisme culturel
Projet Python pour la Data Science de Ribeiro Morgane et Ley Juline

### INTRODUCTION

blablabla

### SOMMAIRE 
* [Installation](#Install)

* [Préparation des données](#prepa)


## Installation <a class="anchor" id="Install"></a>



In [2]:
!pip install -r requirements.txt --quiet

#Importations
import requests
import pandas as pd
import numpy as np
import statsmodels.api as sm
from prince import MCA
from sklearn.cluster import KMeans
from plotnine import *
import geopandas as gpd
import contextily as ctx
from cartiflette import carti_download

#Fonctions
from scripts import fonctions


## Préparation des données <a class="anchor" id="prepa"></a>

Les données utilisées sont également disponibles ici : 

Pratiques d'écoute de musique en ligne - 2020 : https://www.data.gouv.fr/datasets/pratiques-decoute-de-musique-en-ligne-2020 

Consommation des contenus culturels et sportifs numériques - Baromètre : https://www.data.gouv.fr/datasets/consommation-des-contenus-culturels-et-sportifs-numeriques-barometre


### 1. Adresses 

In [5]:
url_barometre = "https://www.data.gouv.fr/api/1/datasets/r/aff9d702-db6d-46e5-8416-c032d8b9d89d"

url_musique = "https://www.data.gouv.fr/api/1/datasets/r/85d1feb0-76cc-4e84-96f4-169971db952e"


### 2. Chargement des données

In [12]:
fonctions.telecharger(url_barometre, "data_barometre.xlsx")
fonctions.telecharger(url_musique, "data_musique.xlsx")

df_barometre_brut = pd.read_excel("data_barometre.xlsx")
df_musique_brut = pd.read_excel("data_musique.xlsx")


Téléchargement réussi !
Téléchargement réussi !


### 3. Structure de la base de données - Baromètre


In [13]:
fonctions.analyser(df_barometre_brut)

'La base de données contient 5002 observations et 831 variables. Elle comprend 180 variables numériques, 651 variables de type texte, On observe 3185977 valeurs manquantes, soit 76.65% de l’ensemble des cellules, réparties sur 736 variables. Enfin, 175 variables sont constantes.'

### 3. Structure de la base de données - Pratique d'écoute de musique en ligne

In [14]:
fonctions.analyser(df_musique_brut)

'La base de données contient 2200 observations et 261 variables. Elle comprend 1 variables numériques, 260 variables de type texte, On observe 214 valeurs manquantes, soit 0.04% de l’ensemble des cellules, réparties sur 1 variables. Enfin, 2 variables sont constantes.'

On se rend compte en ouvrant la base de données que la part de valeurs manquantes est bien plus élevée que 0.04%. Un recodage de deux bases de données est donc nécessaire

### 4. Recodage des données - Baromètre

Le recodage va également nous permettre de trier les variables qui nous intéressent.

In [17]:
variables = [
    "SEXE", "AGE", "AGGLOIFOP2", "TYPCOM", "TAILCOM", "DPT", "REG3", "REG13", "PI4",
    "SITI", "PPIA", "RECPPIA", "STCA", "STATUT", "FOYER", 
    #Q1 quel produit avez-vous consommé de manière dématérialisée ces 12 derniers mois ?
    "Q1_1", "Q1_2", "Q1_3", "Q1_4", "Q1_5", "Q1_6", "Q1_7", "Q1_8", "Q1_9", "Q1_10", "Q1_11", 
    #Q2 : à quelle fréquence ?
    "Q2_r1", "Q2_r2", "Q2_r3", "Q2_r4", "Q2_r5", "Q2_r6", "Q2_r7", "Q2_r8", "Q2_r9", "Q2_r10", "Q2_r11", 
    #Q2BIS : regardez-vous des vidéos de courtes durée proposées par Youtube?
    "Q2BIS",
    #Q3 : consommation légale ? (globalité)
    "Q3",
    #Q4 : consommation légale ? globalité dans le cadre des activités proposées question1
    "Q4",
    #Q5_r : consommation légale pour chaque activité
    "Q5_r1", "Q5_r2", "Q5_r3", "Q5_r4", "Q5_r5", "Q5_r6", "Q5_r7", "Q5_r8", "Q5_r9", "Q5_r10", "Q5_r11", 
    #Q6 : fréquence de conso illégale
    "Q6_r1", "Q6_r2", "Q6_r3", "Q6_r4", "Q6_r5", "Q6_r6", "Q6_r7", "Q6_r8", "Q6_r9", "Q6_r10", "Q6_r11", 
    #QBOL14 : comment découvrez-vous de nouveaux films ?
    "QBOL14_1", "QBOL14_2", "QBOL14_3", "QBOL14_4", "QBOL14_5", "QBOL14_6", "QBOL14_7", "QBOL14_8", "QBOL14_9", 
    #QBU1 : consommation payante ou gratuite ?
    "QBU1_r1", "QBU1_r2", "QBU1_r3", "QBU1_r4", "QBU1_r5", "QBU1_r6", "QBU1_r7", "QBU1_r8", "QBU1_r9", "QBU1_r10", "QBU1_r11", 
    #QBU7 : 3 principales raisons de ne pas consommer légalement
    "QBU7_r1_c1", "QBU7_r2_c1", "QBU7_r3_c1", "QBU7_r4_c1", "QBU7_r5_c1", "QBU7_r6_c1", "QBU7_r7_c1", "QBU7_r8_c1", "QBU7_r9_c1", "QBU7_r10_c1", "QBU7_r11_c1", "QBU7_r12_c1", "QBU7_r13_c1", "QBU7_r14_c1", "QBU7_r15_c1", "QBU7_r16_c1", 
    #RS14 : quel service avez vous accès pour écouter de la musique
    "RS14_1", "RS14_2", "RS14_3", "RS14_4", "RS14_5", "RS14_6", "RS14_7", "RS14_8", "RS14_9"
]

df_barometre_cleaned = df_barometre_brut[variables]

df_barometre_cleaned.replace({"#NUL!": np.nan, "": np.nan, " ": np.nan}, inplace=True)
df_barometre_cleaned = df_barometre_cleaned.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # retirer les espaces au début et à la fin des string pour le recodage au cas où

df_barometre_recode = pd.DataFrame([])

# Sexe
sexe_map = {"Une femme": 0, "Un homme": 1}
df_barometre_recode["sexe"] = df_barometre_cleaned["SEXE"].map(sexe_map)

df_barometre_recode["age"]=df_barometre_cleaned["AGE"]

agglo_map = {"MoinsDe100000Habitants":0, "PlusDe100000Habitants":1}
df_barometre_recode["plusde10000habitants"] = df_barometre_cleaned["AGGLOIFOP2"].map(agglo_map)


df_barometre_recode["commune_centreville"] = np.where(
    df_barometre_cleaned["TYPCOM"] == "VilleCentre", 1, 0)

df_barometre_recode["commune_rural"] = np.where(
    df_barometre_cleaned["TYPCOM"] == "Rural", 1, 0)

df_barometre_recode["commune_banlieue_sup"] = np.where(
    df_barometre_cleaned["TYPCOM"] == "BanlieueNVSup", 1, 0)

df_barometre_recode["commune_banlieue_modeste"] = np.where(
    df_barometre_cleaned["TYPCOM"] == "BanlieueNVModeste", 1, 0)

df_barometre_recode["commune_banlieue_inter"] = np.where(
    df_barometre_cleaned["TYPCOM"] == "BanlieueNVInterm", 1, 0)

df_barometre_recode["commune_isolee"] = np.where(
    df_barometre_cleaned["TYPCOM"] == "VilleIsolee", 1, 0)


df_barometre_recode["taille_commune"] = df_barometre_cleaned["TAILCOM"]

departements_dict = {
    "AIN": "01", "AISNE": "02", "ALLIER": "03", "ALPESDEHAUTESPROVENCE": "04",
    "HAUTESALPES": "05", "ALPESMARITIMES": "06", "ARDECHE": "07", "ARDENNES": "08",
    "ARIEGE": "09", "AUBE": "10", "AUDE": "11", "AVEYRON": "12", "BOUCHESDURHONE": "13",
    "CALVADOS": "14", "CANTAL": "15", "CHARENTE": "16", "CHARENTEMARITIME": "17",
    "CHER": "18", "CORREZE": "19", "CORSES": "2A", "COTEOR": "21", "COTEDARMOR": "22",
    "CREUSE": "23", "DORDOGNE": "24", "DOUBS": "25", "DROME": "26", "EURE": "27",
    "EUREETLOIRE": "28", "FINISTERE": "29", "GARD": "30", "HAUTEGARONNE": "31",
    "GERS": "32", "GIRONDE": "33", "HERAULT": "34", "ILLEETVILAINE": "35",
    "INDRE": "36", "INDREETLOIRE": "37", "ISERE": "38", "JURA": "39", "LANDES": "40",
    "LOIRTCHER": "41", "LOIRE": "42", "HAUTELOIRE": "43", "LOIREATLANTIQUE": "44",
    "LOIRET": "45", "LOT": "46", "LOTETGARONNE": "47", "LOZERE": "48", "MAINEETLOIRE": "49",
    "MANCHE": "50", "MARNE": "51", "HAUTEMARNE": "52", "MAYENNE": "53", "MEURTHEETMOSELLE": "54",
    "MEUSE": "55", "MORBIHAN": "56", "MOSELLE": "57", "NIEVRE": "58", "NORD": "59",
    "OISE": "60", "ORNE": "61", "PASDECALAIS": "62", "PUYDEDOME": "63",
    "PYRENNEESATLANTIQUES": "64", "HAUTESPYRENNEES": "65", "PYRENNEESORIENTALES": "66",
    "BASRHIN": "67", "HAUTRHIN": "68", "RHONE": "69", "HAUTESAONE": "70",
    "SAONEETLOIRE": "71", "SARTHE": "72", "SAVOIE": "73", "HAUTESAVOIE": "74",
    "PARIS": "75", "SEINEMARITIME": "76", "SEINEETMARNE": "77", "YVELINES": "78",
    "DEUXSEVRES": "79", "SOMME": "80", "TARN": "81", "TARNETGARONNE": "82",
    "VAR": "83", "VAUCLUSE": "84", "VENDEE": "85", "VIENNE": "86", "HAUTEVIENNE": "87",
    "VOSGES": "88", "YONNE": "89", "BELFORT": "90", "ESSONNE": "91",
    "HAUTSDESEINE": "92", "SEINESAINTDENIS": "93", "VALDEMARNE": "94", "VALDOISE": "95"
}

df_barometre_recode["dpt"] = df_barometre_cleaned["DPT"].map(departements_dict)

province_map = {"RegionIleDeFrance":0, "PROVINCE":1}
df_barometre_recode["province"] = df_barometre_cleaned["REG3"].map(province_map)

df_barometre_recode["region"] = df_barometre_cleaned["REG13"]

df_barometre_recode["situation_actuelle"] = df_barometre_cleaned["SITI"]

df_barometre_recode["profession"] = df_barometre_cleaned["PPIA"]


df_barometre_recode["csp_plus"] = np.where(
    df_barometre_cleaned["PI4"] == "CSPPLUS", 1, 0)

df_barometre_recode["csp_moins"] = np.where(
    df_barometre_cleaned["PI4"] == "CSPMOINS", 1, 0)

df_barometre_recode["csp_inactifs"] = np.where(
    df_barometre_cleaned["PI4"] == "INACTIFS", 1, 0)

df_barometre_recode["csp_inactifs_plus"] = np.where(
    df_barometre_cleaned["RECPPIA"] == "RetraitesCSPPlus", 1, 0)

df_barometre_recode["csp_inactifs_moins"] = np.where(
    df_barometre_cleaned["RECPPIA"] == "RetraitesCSPMoins", 1, 0)


df_barometre_recode["foyer"] = df_barometre_cleaned["FOYER"]

df_barometre_recode[["conso_demat_mus", "conso_demat_films", "conso_demat_series", "conso_demat_photos", "conso_demat_jv", "conso_demat_livres", "conso_demat_logi", "conso_demat_presse", "conso_demat_retrans"]] = df_barometre_cleaned[["Q1_1", "Q1_2", "Q1_3", "Q1_4", "Q1_5", "Q1_6", "Q1_7", "Q1_8", "Q1_9"]]
df_barometre_recode[["conso_demat_mus", "conso_demat_films", "conso_demat_series", "conso_demat_photos", "conso_demat_jv", "conso_demat_livres", "conso_demat_logi", "conso_demat_presse", "conso_demat_retrans"]] = df_barometre_recode[["conso_demat_mus", "conso_demat_films", "conso_demat_series", "conso_demat_photos", "conso_demat_jv", "conso_demat_livres", "conso_demat_logi", "conso_demat_presse", "conso_demat_retrans"]].applymap(lambda x: 1 if isinstance(x, str) and x.strip() != "" else 0)

df_barometre_recode[["freq_demat_mus", "freq_demat_films", "freq_demat_series", "freq_demat_photos", "freq_demat_jv", "freq_demat_livres", "freq_demat_logi", "freq_demat_presse", "freq_demat_retrans"]] = df_barometre_cleaned[["Q2_r1", "Q2_r2", "Q2_r3", "Q2_r4", "Q2_r5", "Q2_r6", "Q2_r7", "Q2_r8", "Q2_r9"]]

df_barometre_recode["conso_legale"] = np.where(
    df_barometre_cleaned["Q4"] == "Uniquement de manière légale", 1, 0)

df_barometre_recode["conso_illegale"] = np.where(
    df_barometre_cleaned["Q4"] == "Uniquement de manière illégale", 1, 0)

mix_legal = [
    "Généralement de manière légale même s’il peut m’arriver de le faire de manière illégale",
    "Autant de manière légale qu’illégale",
    "Généralement de manière illégale même s’il peut m’arriver de le faire de manière légale"
]

df_barometre_recode["conso_legale_et_illegale"] = (
    df_barometre_cleaned["Q4"].isin(mix_legal)
).astype(int)



print(df_barometre_recode.head())





   sexe  age  plusde10000habitants  commune_centreville  commune_rural  \
0     0   25                     0                    1              0   
1     1   64                     0                    0              0   
2     0   35                     0                    0              1   
3     0   65                     1                    1              0   
4     1   40                     1                    1              0   

   commune_banlieue_sup  commune_banlieue_modeste  commune_banlieue_inter  \
0                     0                         0                       0   
1                     0                         0                       1   
2                     0                         0                       0   
3                     0                         0                       0   
4                     0                         0                       0   

   commune_isolee     taille_commune  ...          freq_demat_series  \
0               0   

/tmp/ipykernel_34696/2664599049.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_34696/2664599049.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
/tmp/ipykernel_34696/2664599049.py:122: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.


### 4. Recodage des données - Pratique d'écoute de musique en ligne

In [ ]:
#df[["var_fr", "poprock", "rap", "classique", "jazz", "dance", "electro", "metal", "rnb", "soul", "reggae", "musique_monde"]] 
